### Machine Learning with Pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.host", "localhost").appName('Filter').getOrCreate()

In [35]:
spark.read.csv("C:/Users/gorja/Downloads/test1.csv",header=True,inferSchema=True)

DataFrame[Name: string, age: int, Experience: int, Salary: int]

In [3]:
training = spark.read.csv("C:/Users/gorja/Downloads/test1.csv",header=True,inferSchema=True)

In [4]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [9]:
output = featureassembler.transform(training)


In [10]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data = output.select("Independent Features","Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [32]:
from pyspark.ml.regression import LinearRegression
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [16]:
### coeffients
regressor.coefficients

DenseVector([-115.8558, 1602.7301])

In [18]:
### Intercepts
regressor.intercept

16904.795239761766

In [33]:
pred_results = regressor.evaluate(train_data)


In [34]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|20000.000000000073|
|          [29.0,4.0]| 20000|19999.999999999927|
|          [30.0,8.0]| 25000|25000.000000000007|
+--------------------+------+------------------+

